## 코딩테스트

### MNIST 손글씨 숫자 분류

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### 데이터 로딩 및 전처리

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'CPU/GPU: {device}')

transform = transforms.ToTensor()

train_set = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform)

test_set = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

CPU/GPU: cuda


100%|██████████| 9.91M/9.91M [00:01<00:00, 6.48MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 155kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.41MB/s]
100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]


### CNN 모델 선언

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # 문제. 아래의 주석을 풀고 알맞은 Flatten 내용을 추가하시오.
        # x = _________________________________
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        #x = torch.sigmoid(self.fc2(x))
        return x

In [28]:
model = CNN()

### 손실함수와 옵티마이저

In [29]:
criterion = nn.CrossEntropyLoss()
# 문제: 아래의 주석을 해제, Adam 옵티마이저를 설정하고 학습률은 0.001로 지정하세요.
# optimizer = _________________________________
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### 학습

In [30]:
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        outputs = model(images)
        # 문제: 아래의 주석 해제, 손실(loss)를 계산하세요.
        # loss = _________________________________
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"[Epoch {epoch+1}] Loss: {running_loss / len(train_loader):.4f}")

[Epoch 1] Loss: 0.2389
[Epoch 2] Loss: 0.0710
[Epoch 3] Loss: 0.0498
[Epoch 4] Loss: 0.0381
[Epoch 5] Loss: 0.0303


### 평가 및 저장

In [34]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)    # 확귤이 가장 높은 클래스 인덱스 선택
        total += labels.size(0) # 전체 데이터 수 누적
        # 문제: 주석해제 후, 정확히 예측한 개수를 누적하세요.
        # correct += _________________________________
        correct += (predicted == labels).sum().item() # predicted -> 모델이 예측한 클래스 / labels -> 실제정답 클래스

print(f'테스트 정확도: {100 * correct / total:.2f}%')

# 문제: 모델 저장 (선택) - 학습된 모델을 'mnist_cnn.pth' 파일로 저장하는 코드를 작성하시오
# torch.save(____________________, 'mnist_cnn.pth')
torch.save(model.state_dict(),'mnist_cnn.pth')
model.eval()

테스트 정확도: 98.99%


CNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

#### 위의 내용을 CUDA에서 동작하도록 하나의 셀에 작성 완성하시오

In [36]:
# 문제: 전체 소스 작성

import torch
import torch.nn as nn
import torch.nn.functional as F

#1 CNN 모델 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # 문제. 아래의 주석을 풀고 알맞은 Flatten 내용을 추가하시오.
        # x = _________________________________
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x    
        

#2 cuda 준비(또는 cpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

#3 모델, 손실 함수, 옵티마이저 정의 및 device로 이동 
model = CNN().to(device)    # a모델을 GPU 또는 CPU에 올림
criterion = nn.CrossEntropyLoss() # 다중 클래스 분류용 손실함수
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # Adam 옵티마이저


#4 학습루프 시작 
epochs = 5  # 에폭 수 설정
for epoch in range(epochs):
    model.train()  # 모델을 학습 모드로 전환
    running_loss = 0.0  # 에폭마다 손실 누적 초기화

    for images, labels in train_loader:  # 훈련 데이터 반복
        images = images.to(device)  # 이미지 텐서를 GPU로 이동
        labels = labels.to(device)  # 라벨도 GPU로 이동

        outputs = model(images)  # 모델에 입력 전달 → 예측 출력
        loss = criterion(outputs, labels)  # 예측 결과와 정답으로 손실 계산

        optimizer.zero_grad()  # 이전 gradient 초기화
        loss.backward()        # 역전파로 gradient 계산
        optimizer.step()       # 가중치 업데이트

        running_loss += loss.item()  # 현재 배치 손실 누적

    # 한 에폭이 끝날 때 평균 손실 출력
    print(f"[Epoch {epoch+1}] Loss: {running_loss / len(train_loader):.4f}")

#5 평가루프
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)   # 입력 이미지 GPU
        labels = labels.to(device)  # 라벨도 GPU로 
        
        outputs = model(images) # 모델도 GPU에서 연산
        _, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct/total
print(f'테스트 정확도: {accuracy: 2f}%')


#6 모델 저장
torch.save(model.state_dict(),'mnist_cnn.pth')

[Epoch 1] Loss: 0.2324
[Epoch 2] Loss: 0.0713
[Epoch 3] Loss: 0.0488
[Epoch 4] Loss: 0.0381
[Epoch 5] Loss: 0.0300
테스트 정확도:  98.720000%
